In [1]:
# Activate Conda Env "Ultra" As the Kernel

import torch
import pandas as pd
from explainers.ego_util import batched_k_hop_subgraph
import networkx as nx
from pyvis.network import Network


In [9]:
import easydict
cfg = {'dataset':{'class': 'synthetic', 'root': '/storage/ryoji/Graph-Transformer/NBFNet-PyG/datasets/hetero_dataset/'}}
cfg = easydict.EasyDict(cfg)

In [13]:
dataset = util.build_dataset(cfg)

ntype2id, rel2id = dataset.ntype2id, dataset.rel2id

id2relation = {}
for k, v in rel2id.items():
    id2relation[v] = k
    
# id2entity, id2relation = torch.load('/storage/ryoji/Graph-Transformer/NBFNet-PyG/wn18rr_id2name.pt')

synthetic dataset
#train: 499, #valid: 71, #test: 144


In [50]:
split = 'valid' # the split
ratio = 0.001 # the top r ratio of edges

if split == 'valid':
    data_index = 1
if split == 'test':
    data_index = 2

# output_file = f'/storage/ryoji/Graph-Transformer/NBFNet-PyG/explanation/NBFNet/WN18RR/explanation_output/{split}_output_hard_edge_mask_top_ratio_{ratio}.pt'
output_file = '/storage/ryoji/Graph-Transformer/NBFNet-PyG/RelPGExplainer/NBFNet/synthetic/2024-12-20-11-35-50/valid_output_hard_edge_mask_top_k_100.pt'
# explanation_file = f'/storage/ryoji/Graph-Transformer/NBFNet-PyG/explanation/NBFNet/WN18RR/explanation_output/{split}_explanations_hard_edge_mask_top_ratio_{ratio}.pt'
explanation_file = '/storage/ryoji/Graph-Transformer/NBFNet-PyG/RelPGExplainer/NBFNet/synthetic/2024-12-20-11-35-50/valid_explanations_hard_edge_mask_top_k_100.pt'
explanations = torch.load(explanation_file)
outputs = torch.load(output_file)
output_df = pd.DataFrame(outputs)

data = dataset[data_index]

In [51]:
def vizualize_explanation(index):
    row = output_df.iloc[index]
    if row['Mode'] == 1:
        rel = id2relation[row['Rel']]
    else:
        rel = id2relation[row['Rel']]+'_inv'

    head_id = row['Heads']
    tail_id = row['Tails']
    head = head_id
    tail = tail_id

    print(f"*** Query: {head, rel}. Answer: {tail}, Rank given the explanation: {row['Ranking']} ***")
    nodes, edges = batched_k_hop_subgraph(torch.tensor([head_id]).unsqueeze(0), 6, data.edge_index, data.num_nodes)

    print(f'Is the tail included in the 6-hop neighbor of head in the original graph? {nodes[0, tail_id].item()}')
    full_edge_index = data.edge_index[:, edges.squeeze()]

    expl = explanations[index]
    # Check that the explanation are restricted only to within the ego_network (not necessary)
    # assert torch.all(torch.logical_and(expl, ~edges.squeeze()) == False)
    expl_edge_index = data.edge_index[:, expl.to(torch.bool)]

    print(f'Is the head included in the Explanation? {torch.any((expl_edge_index == head_id)).item()}')
    print(f'Is the tail included in the Explanation? {torch.any((expl_edge_index == tail_id)).item()}')

    # Create a directed graph
    G = nx.DiGraph()

    # Add edges from expl_edge_index
    for i in range(expl_edge_index.shape[1]):
        source = expl_edge_index[0, i].item()
        target = expl_edge_index[1, i].item()
        # G.add_edge(source, target,arrows='to')
        G.add_edge(source, target)

    attrs = {head_id: "#FF0000", tail_id: "#00FF00"}
    nx.set_node_attributes(G, attrs, name='color')

    try:
        paths = list(nx.all_simple_paths(G, source = head_id, target=tail_id, cutoff = 6))
    
        attrs = {}
        if len(paths) > 0:
            connected = True
        else:
            connected = False
    
        print(f'Is the tail connected to the head? {connected}')
        for path in paths:
            src = head_id
            for node in path[1:]:
                edge = (src, node)
                attrs[edge] = {'color':"#b27ebd"}
                src = node

        nx.set_edge_attributes(G, attrs)
    except:
        print(f'Is the tail connected to the head? {False}')
            
    
    return G
    

In [52]:
display(output_df)

,Ranking,Heads,Tails,Rel,Mode,Num_Edges,Num_Nodes,Inclusion
0,15,158,138,4,1,100,87,1
1,4,222,118,4,1,100,88,1
2,17,190,106,4,1,100,88,0
3,27,203,121,4,1,100,88,0
4,38,224,104,4,1,100,89,1
...,...,...,...,...,...,...,...,...
66,28,183,125,4,1,100,91,0
67,1,184,136,4,1,100,87,0
68,3,198,98,4,1,100,88,1
69,21,171,87,4,1,100,87,0


In [69]:
# Control the index (row number of output_df of the instance you want to inspect here)
index = 5

In [70]:
G = vizualize_explanation(index)

*** Query: (223, 'likes'). Answer: 130, Rank given the explanation: 16 ***
Is the tail included in the 6-hop neighbor of head in the original graph? True
Is the head included in the Explanation? True
Is the tail included in the Explanation? True
Is the tail connected to the head? False


In [71]:
# *** Explanation Graph Visualization ***
# RED NODE: The Query Head (if inside the explanation)
# GREEN NODE: The Answer Tail (if inside the explanation)
# Purple Edges: Edges that are in any path that connected Red to Green
net = Network(notebook = True, cdn_resources = "remote",
                    bgcolor = "#222222",
                    font_color = "white",
                    height = "750px",
                    width = "100%",
                    select_menu = True,
                    filter_menu = True,
                    directed=True
    )
net.from_nx(G)
net.inherit_edge_colors(False)
net.set_edge_smooth('dynamic')
net.show('graph.html')

graph.html
